In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
         fd = os.path.join(dirname, filename)
        
df = pd.read_csv(fd)
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print(df.info())
df.head(10)

* **Since P/E has 2 rows Nan and P/B has 8 rows Nan, we will clean them so that we can have smooth and consistent dataset**
* **Also, SEC Filings are of no use to us for analysis**


In [ ]:
df.drop(df[df['Price/Earnings'].isna() | df['Price/Book'].isna()].index, inplace=True)
df.drop(columns='SEC Filings', inplace=True)
df.drop(columns='Dividend Yield', inplace=True)
df.info()

# SECTOR_WISE

In [ ]:
sectorwise = df.groupby(['Sector']).agg(np.size)
sectorwise.drop(sectorwise.columns[1:], axis=1, inplace=True)
sectorwise.columns  = ['No. of Companies']
# sectorwise.rename(columns={sectorwise.columns[0] : 'No. of Companies'}, inplace = True)
sectorwise


In [ ]:
df.head(10)

In [ ]:
sectorwise = df.groupby(['Sector']).agg(np.sum)
sectorwise['total_shares'] = abs(sectorwise['Market Cap']/sectorwise['Earnings/Share'])
sectorwise

# sectorwise.drop(sectorwise.columns[1:], axis=1, inplace=True)
# sectorwise.columns  = ['No. of Companies']
# # sectorwise.rename(columns={sectorwise.columns[0] : 'No. of Companies'}, inplace = True)
# sectorwise


In [ ]:
sectorwise = df.groupby(['Sector']).agg(np.min)
sectorwise

In [ ]:
sectorwise = df.groupby(['Sector']).agg(np.max)
sectorwise

In [ ]:
sectorwise = df.groupby(['Sector']).agg(np.max)
sectorwise

In [ ]:
sectorwise = df[df['Sector'] == 'Consumer Discretionary']
sectorwise.iloc[sectorwise['Earnings/Share'].argmax()]

In [ ]:
sectorwise.iloc[sectorwise['Earnings/Share'].argmin()]

In [ ]:
sectorwise['Earnings/Share'].mean()

In [ ]:
sectorwise['Earnings/Share'].std()

In [ ]:
sectorwise[sectorwise['Earnings/Share'] > 0].std()

In [ ]:
sectorwise[sectorwise['Earnings/Share'] < 0].std()

In [ ]:
print(sectorwise['Price'].max(),sectorwise['Price'].min(),sectorwise['Price'].mean(),sectorwise['Price'].std())

In [ ]:
pricemax = round(sectorwise['Price'].max())
pricemin = round(sectorwise['Price'].min())
pricedev = round(sectorwise['Price'].mean())

print(pricemax,pricemin)

pricerange = []

while True:
    if(pricemin <= pricemax):
        range1 = pricemin
        pricemin += pricedev
        range2 = pricemin
        pricerange.append(tuple((range1,range2)))
    else:
        break

In [ ]:
pricerange

In [ ]:
for range1,range2 in pricerange:
    print("(",range1,",",range2,")" , sectorwise[(sectorwise['Price'] >= range1) & (sectorwise['Price'] <= range2)]['Price'].mean(),len(sectorwise[(sectorwise['Price'] >= range1) & (sectorwise['Price'] <= range2)]['Price']))

In [ ]:
print(format("range1","5s"),format("-","10s"),format("range2","5s"),format("=>","5s"), format("avg","5s"),format("maxp","5s"),format("minp","5s"),format("earningsum","5s"),format("earningavg","5s"))
for range1,range2 in pricerange:
    lowprice = sectorwise[(sectorwise['Price'] >= range1) & (sectorwise['Price'] <= range2)]['52 Week Low'].mean()
    highprice = sectorwise[(sectorwise['Price'] >= range1) & (sectorwise['Price'] <= range2)]['52 Week High'].mean()
    earnings = sectorwise[(sectorwise['Price'] >= range1) & (sectorwise['Price'] <= range2)]['Earnings/Share'].sum()
    earningsavg = sectorwise[(sectorwise['Price'] >= range1) & (sectorwise['Price'] <= range2)]['Earnings/Share'].mean()
    if lowprice > 0 and highprice > 0:
        print(range1,"-",range2,"=>" , format(sectorwise[(sectorwise['Price'] >= range1) & (sectorwise['Price'] <= range2)]['Price'].mean(),".2f") ,format(lowprice,".2f"), format(highprice,".2f"),format(earnings,".2f"),format(earningsavg,".2f"), sep="\t")

In [ ]:
def givePriceRange(sector):
    pricemax = round(sector['Price'].max())
    pricemin = round(sector['Price'].min())
    pricedev = round(sector['Price'].mean())
    
    print("Max Price - ", pricemax,"\tMin Price - ",pricemin,"\tMean Price - ",pricedev)

    pricerange = []

    while True:
        if(pricemin <= pricemax):
            range1 = pricemin
            pricemin += pricedev
            range2 = pricemin
            pricerange.append(tuple((range1,range2)))
        else:
            break
            
    return pricerange

In [ ]:
def sector_range(sector,sectorname):
    print("Sector - ",sectorname)
    print("\n")
    
    pricerange = givePriceRange(sector)
    print("\n")
    
    for range1,range2 in pricerange:
        print("(",range1,",",range2,")" , sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Price'].mean(),len(sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Price']))
    
    print("\n")
    print(format("range1","5s"),format("-","6s"),format("range2","9s"),format("=>","7s"), format("avg","7s"),format("maxp","7s"),format("minp","5s"),format("earningsum","5s"),format("earningavg","5s"))
    for range1,range2 in pricerange:
        lowprice = sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['52 Week Low'].mean()
        highprice = sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['52 Week High'].mean()
        earnings = sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Earnings/Share'].sum()
        earningsavg = sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Earnings/Share'].mean()
        if lowprice > 0 and highprice > 0:
            print(range1,"-",range2,"=>" , format(sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Price'].mean(),".2f") ,format(lowprice,".2f"), format(highprice,".2f"),format(earnings,".2f"),format(earningsavg,".2f"), sep="\t")
    
    print("---------------------------------------------")

In [ ]:
for sector in df.groupby(df['Sector']).agg(np.size).index:
    sector_range(df[df['Sector'] == sector], sector)

# Price -> Price/Earnings

In [ ]:
def sector_range_2(sector,sectorname):
    print("Sector - ",sectorname)
    print("\n")
    
    pricerange = givePriceRange(sector)
    print("\n")
    
    
    print("\n")
    print(format("range1","5s"),format("-","6s"),format("range2","9s"),format("=>","7s"), format("avg","7s"),format("Price/Earnings","5s"))
    for range1,range2 in pricerange:
        pe = sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Price/Earnings'].mean()
        print(range1,"-",range2,"=>" , format(sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Price'].mean(),".2f") ,format(pe,".2f"), sep="\t")
    
    print("---------------------------------------------")

In [ ]:
for sector in df.groupby(df['Sector']).agg(np.size).index:
    sector_range_2(df[df['Sector'] == sector], sector)

In [ ]:
df['total_sale'] = df['Market Cap']/df['Price/Sales']

In [ ]:
def sector_range_3(sector,sectorname):
    print("Sector - ",sectorname)    
    pricerange = givePriceRange(sector)
    print("\n")
    print(format("range1","5s"),format("-","6s"),format("range2","9s"),format("=>","7s"), format("avg","7s"),format("total_sale","5s"),format("Price/Sales","5s") )
    for range1,range2 in pricerange:
        pe = sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['total_sale'].mean()
        psrt = sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Price/Sales'].mean()
        print(range1,"-",range2,"=>" , format(sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Price'].mean(),".2f") ,format(pe,".2f"),format(psrt,".2f"), sep="\t")
    print("---------------------------------------------")

In [ ]:
for sector in df.groupby(df['Sector']).agg(np.size).index:
    sector_range_3(df[df['Sector'] == sector], sector)

# Price -> Price/Book

In [ ]:
def sector_range_4(sector,sectorname):
    print("Sector - ",sectorname)
    print("\n")
    
    pricerange = givePriceRange(sector)
    print("\n")
    
    
    print("\n")
    print(format("range1","5s"),format("-","6s"),format("range2","9s"),format("=>","7s"), format("avg","7s"),format("Price/Book","5s"))
    for range1,range2 in pricerange:
        pe = sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Price/Book'].mean()
        print(range1,"-",range2,"=>" , format(sector[(sector['Price'] >= range1) & (sector['Price'] <= range2)]['Price'].mean(),".2f") ,format(pe,".2f"), sep="\t")
    
    print("---------------------------------------------")

In [ ]:
for sector in df.groupby(df['Sector']).agg(np.size).index:
    sector_range_4(df[df['Sector'] == sector], sector)

# Visualization

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns


%matplotlib inline

In [ ]:
df.columns

In [ ]:
stockpricedf = df.groupby('Sector').agg(np.sum)
stockpricedf = stockpricedf.reset_index()

In [ ]:
sectors = stockpricedf.index
prices  = stockpricedf['Price']


plt.figure(figsize= (10, 10))
plt.bar(sectors,prices,0.35)
plt.show()

In [ ]:
stockpricemax = df.groupby('Sector').agg(np.max)
stockpricemin = df.groupby('Sector').agg(np.min)

sectors = stockpricemax.index
mprices  = stockpricemax['Price']
minprices  = stockpricemin['Price']

x = np.arange(len(sectors))
width=0.35

fig, ax = plt.subplots(figsize= (10, 10))
rect1 = ax.bar(x - width/2, mprices, width, label='Max')
rect2 = ax.bar(x + width/2, minprices, width, label='Min')
ax.set_xticks(x)
ax.set_xticklabels(sectors)
# ax.bar_label(rects1,mprices, padding=3)
# ax.bar_label(rects2,minprices,padding=3)
ax.legend()
plt.show()

In [ ]:
stockpriceavg = df.groupby('Sector').agg(np.mean)

sectors = stockpriceavg.index
prices  = stockpriceavg['Price']


plt.figure(figsize= (10, 10))
plt.bar(sectors,prices,0.35)
plt.show()

In [ ]:
plt.figure(figsize= (20, 20))


ax = sns.violinplot(x="Sector", y="Price", data=df, palette="muted")

In [ ]:
sns.pairplot(df)


In [ ]:
plt.figure(figsize= (16, 16))

sns.heatmap(df.corr())

In [ ]:
# df[stockpricedf.Sector] = 0
nwdf = pd.get_dummies(df,columns = ['Sector'])
nwdf.head()

# Stock Price Prediction

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X, y = nwdf[nwdf.columns[3:]], nwdf['Price']
clf = LinearRegression()
clf.fit(X, y)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
clf.score(X_test, y_test)


In [ ]:
X_test.iloc[0]

X_test.iloc[0]['Price/Earnings'] = 24

d = pd.DataFrame(X_test.iloc[0])
d= d.transpose()
d

clf.predict(d)